<a href="https://colab.research.google.com/github/georgiivarygin/weapon_detection_ISS_HACK/blob/main/iss_hack_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключаем colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Импортируем библиотеки

In [ ]:
import cv2
import matplotlib.pyplot as plt

import glob
import os
import shutil

from tqdm.notebook import tqdm

from moviepy.editor import * 
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2015232/45929032 bytes (4.4%)5545984/45929032 bytes (12.1%)8896512/45929032 bytes (19.4%)12443648/45929032 bytes (27.1%)15917056/45929032 bytes (34.7%)19472384/45929032 bytes (42.4%)22675456/45929032 bytes (49.4%)26288128/45929032 bytes (57.2%)29794304/45929032 bytes (64.9%)32538624/45929032 bytes (70.8%)36192256/45929032 bytes (78.8%)39763968/45929032 bytes (86.6%)

# Устанавливаем необходимые библиотеки для работы yolov5

In [ ]:
%cd /content/drive/MyDrive/iss_hack/yolov5
%pip install -r requirements.txt # Установим необходимые библиотеки

# Стандартная функция по обрезке видео работает криво, напишем свою функцию

In [ ]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

ffmpeg_extract_subclip("/content/drive/MyDrive/iss_hack/weapon_train.mp4", 0, 15, targetname="/content/drive/MyDrive/iss_hack/test_0_15.mp4")


[MoviePy] Running:
>>> /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1 -y -ss 0.00 -i /content/drive/MyDrive/iss_hack/weapon_train.mp4 -t 15.00 -vcodec copy -acodec copy /content/drive/MyDrive/iss_hack/test_0_15.mp4
... command successful.


# Тестируем сеть

In [ ]:
# Параметр —conf-thres позволяет установить пороговое значение для confidence модели.
# Все объекты, confidence которых ниже этого значения не считаются объектами.
# Дополнительно редактировался файл examintion_train.py так как по условиям необходимо было разметить видео: на какой секунде есть оружие.

!python /content/drive/MyDrive/iss_hack/yolov5/detect.py --weights /content/drive/MyDrive/iss_hack/best.pt --conf-thres 0.65 --source /content/drive/MyDrive/iss_hack/test_0_15.mp4

detect: weights=['/content/drive/MyDrive/iss_hack/best1.pt'], source=/content/drive/MyDrive/iss_hack/test_0_15.mp4, data=drive/MyDrive/iss_hack/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.65, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=drive/MyDrive/iss_hack/yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-207-g8efe977 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1802) /content/drive/MyDrive/iss_hack/test_0_15.mp4: 384x640 Done. (0.027s)
FPS of video 120.0
video 1/1 (2/1802) /content/drive/MyDrive/iss_hack/test_0_15.mp4: 384x640 Done. (0.027s)
video 1/1 (3/1802) /content/drive/MyDrive/iss_hack/test_0_15.mp4: 384x640 Done. 

# Функция для записи файла с посекундной разметкой видео на предмет присутствия/отсутствия оружия 

In [ ]:
%time
import pandas as pd
from moviepy.editor import VideoFileClip
import numpy as np
df = pd.read_csv('/content/log_file.csv', header=None)
clip = VideoFileClip("/content/drive/MyDrive/iss_hack/weapon_train.mp4")
df['length'] = clip.duration / len(df)
df['duration'] = np.floor(df['length'].cumsum())
df_group = df.groupby(['duration'], as_index=False).agg({
        1: 'max',
        4: 'sum'})
%cd /content
with open('weapon.txt', 'w') as f:
  for i in range(len(df_group)-1):
    f.write('{} {} {} \n'.format(i, i+1, df_group[1][i]))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
/content
